In [55]:
# Import all the libraries needed

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader


import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.4.0
Torchvision Version:  0.5.0


In [56]:
# Define the transformations for the data
array_transforms = {
    # Training data is augmented
    'train':
    transforms.Compose([
#        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=180),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
#        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
#        transforms.Normalize([0.485, 0.456, 0.406],
#                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation data is not augmented
    'val':
    transforms.Compose([
#        transforms.Resize(size=256),
#        transforms.CenterCrop(size=224),
        transforms.RandomRotation(degrees=180),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
#        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# Tell the dataloader where to search
traindir = 'Data/jAER_simplest_by_hand_better/traindir'
valdir = 'Data/jAER_simplest_by_hand_better/valdir'
testdir = 'Data/jAER_simplest_by_hand_better/testdir'

train_r = 'Data/jAER_simplest_by_hand_better/traindir/r_data'
train_g = 'Data/jAER_simplest_by_hand_better/traindir/g_data'

#traintensors = []
#valtensors = []

# npy_example = np.load('Data/jAER_simplest_by_hand_better/traindir/r_data/r_data-0000.npy')
# traintensors.append(npy_example)

train_inputs = np.load('Data/jAER_simplest_by_hand_better/traindir/r_data/r_data-0000.npy')
train_labels = np.load('Data/jAER_simplest_by_hand_better/traindir/labels/label-0.npy')
batch_size = 8

class CreateDataset(Dataset):
    def __init__(self, train_inputs, train_labels, transform=None):
        self.train_inputs = train_inputs
        self.train_labels = torch.LongTensor(train_labels)
        self.transform = transform

    def __getitem__(self, index):
        x = self.train_inputs[index]
        y = self.train_labels[index]

        if self.transform:
            x = Image.fromarray(self.train_inputs[index].astype(np.uint8).transpose(1,2,0))
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.train_inputs)


# Get the data as Tensors
data = {
   'train':
   CreateDataset(train_inputs, train_labels, transform = array_transforms['train']),
#    'val':
#    CreateDataset(val_inputs, val_labels, transform = array_transforms['val']),
}

# Load Data in batches, shuffled
dataloaders = {
   'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
#    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
}

In [57]:
# Load in pretrained VGG16 net and extract the fully convolutional part
vgg16 = models.vgg16(pretrained=True)
vgg16_mod = vgg16.features
print(vgg16_mod)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si